In [ ]:
#%pylab inline
#%config InlineBackend.figure_format = 'retina' 

In [1]:
#here are some common modules:
import scipy as sp #library of scientific functions
import scipy.io 
import scipy.signal as signal
import numpy as np #library of math functions
import pandas as pd #library of data analysis functions
import matplotlib.pyplot as plt #functions to plot data
import os #This lets python talk to your opperating system to open and save files.
from parabolic import parabolic
import matplotlib.mlab as mlab
from mpl_toolkits.mplot3d import Axes3D

In [2]:
filename = 'emodat.mat' #adjut file name here
filename = os.path.join('../FM-BCI', filename) #adjust filepath 
datafile = sp.io.loadmat(filename) #loading filename
#print datafile.keys()
voltageSamples = datafile['data'] 
#print voltageSamples.shape, len(voltageSamples)

In [3]:
variances = []
means = []
medians = []
standardDeviations = []
for i in range(len(voltageSamples)):
    variances.append(np.var(voltageSamples[i,:]))
    means.append(np.mean(voltageSamples[i,:]))
    medians.append(np.median(voltageSamples[i,:]))
    standardDeviations.append(np.std(voltageSamples[i,:]))

In [4]:
for i in range(len(voltageSamples)):
    print 'Channel', i+1
    print '\tmean:', means[i]
    print '\tmedian:', medians[i]
    print '\tstandard deviation:', standardDeviations[i]
    print '\tvariance:', np.var(voltageSamples)

Channel 1
	mean: 53.5051604742
	median: 8.42684103561
	standard deviation: 105.317167517
	variance: 11091.7057739


In [5]:
fig1 = plt.figure(1, figsize=(9,6))
numBins = 50
n, bins, patches = plt.hist(voltageSamples[0],numBins,alpha=0.8)


In [7]:
fig2 = plt.figure(2, figsize=(9,6))
plt.plot(voltageSamples[0])
plt.show()

In [8]:
sampleRate = 512 # sample rate assumed during recording
sampleSpacing = 1.0 / sampleRate # time between samples in seconds
dataLengthSecs = 5 # length of whole recording
dataLengthSamples = dataLengthSecs*sampleRate # length of whole recording in samples

t = np.arange(0,dataLengthSecs,sampleSpacing) # time vector spanning length in seconds, with approp. num of samples
#numOfChannel = voltageSamples.shape[0] # determine number of channels from data, i.e. not predefined
numOfChannel = 8
voltageSamples = np.empty([numOfChannel, dataLengthSamples])

# channel weighting
channelWeights = np.linspace(1,1./numOfChannel,numOfChannel) 
#np.random.shuffle(channelWeights)
print 'Channel weights:', '\n'
for channelIndex in range(numOfChannel):
    print "        ", channelIndex+1, "   ", channelWeights[channelIndex]
    
# frequency modulation parameters
alphaCenter = 10.25   # Hz the carrier frequency
alphaModFreq = 1  # Hz the modulating frequency
alphaFreqDev = 4   # Hz of the frequency deviation

# signal to noise parameters
snr = 2             # signal / noise
noiseMean = 0
noiseStdDev = 0.5
alphaMean = 0
alphaStdDev = abs(np.sqrt(snr*(noiseStdDev**2))) # std of sine wave
alphaAmp = np.sqrt(2)*alphaStdDev
h = alphaFreqDev/alphaModFreq         # Modulation index

# Constructs 1/f noise by iteratively adding normal random noise, effectively the CDF of normal dist.
normalNoise = np.random.normal(noiseMean, noiseStdDev, (1,dataLengthSamples))
pinkNoise = np.cumsum(normalNoise)

# Frequency modulated alpha rhythm, a sinusoidal baseband signal
alpha = alphaAmp*np.sin( alphaCenter  * 2.0 * np.pi * t + alphaFreqDev*np.sin(2 * np.pi * alphaModFreq * t) / alphaModFreq)

# assign the weighted alpha rhythm + 1/f noise + additional random noise to each channel in sample
for channelIndex in range(0,numOfChannel):
    voltageSamples[channelIndex,:] = channelWeights[channelIndex]*alpha + pinkNoise + 0.5*np.random.random([1,dataLengthSamples])    
voltageSamples[channelIndex,:] = alpha # change last channel to ground truth alpha

fig3 = plt.figure(3, figsize=(9,6))

channelArray = np.vsplit(voltageSamples,1)
for channelIndex in range(0,numOfChannel):
    plt.plot(voltageSamples[channelIndex,1:sampleRate])
plt.show()


Channel weights: 

         1     1.0
         2     0.875
         3     0.75
         4     0.625
         5     0.5
         6     0.375
         7     0.25
         8     0.125


In [26]:
def butter_bandpass(lowcut, highcut, fs, order=4):
        #lowcut is the lower bound of the frequency that we want to isolate
        #hicut is the upper bound of the frequency that we want to isolate
        #fs is the sampling rate of our data
        nyq = 0.5 * fs #nyquist frequency - see http://www.dspguide.com/ if you want more info
        low = float(lowcut) / nyq
        high = float(highcut) / nyq
        b, a = sp.signal.butter(order, [low, high], btype='band')
        return b, a

def butter_bandpass_filter(mydata, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sp.signal.filtfilt(b, a, mydata)
    return y

winLengthSecs = 1 # predefine length of window.
winLengthSamples = winLengthSecs*sampleRate # length of window in samples
numOfWindows = int(dataLengthSamples/winLengthSamples) # determine number of windows
channelPeaks = np.empty([numOfChannel, numOfWindows]) # container for peak frequencies for each channel every second
desiredFreqResolution = 0.1 # predefine resolution of spectrum
fftLengthSamples = int(sampleRate/desiredFreqResolution)
nyq = 0.5*sampleRate # maximum possible frequency to measure
freqs = scipy.fftpack.rfftfreq(fftLengthSamples,sampleSpacing) # retrieve frequency axis

bandLow = 9.5                                # lower alpha band 
bandHigh = 10.5                              # higher alpha band
orderFilter = 4  

arrayOfAmps = np.empty([numOfChannel, numOfWindows, len(freqs)])

for channelIndex in range(0,numOfChannel):
    for winIndex in range(0,numOfWindows):
        channelVoltage = voltageSamples[channelIndex,:]
        channelVoltageFilt = butter_bandpass_filter(channelVoltage, bandLow, bandHigh,sampleRate,orderFilter)
            
        # get next window of data, detrend
        winStart = int(winIndex*winLengthSecs*sampleRate)
        winEnd = int((winIndex+1)*winLengthSecs*sampleRate)
        channelVoltageWin = channelVoltageFilt[winStart:winEnd] - np.mean(channelVoltageFilt[winStart:winEnd])
        
        # window next window of data
        winLengthSamples = len(channelVoltageWin) # determine window length
        #windowed = channelVoltage * signal.blackmanharris(winLengthSamples)
        windowedWin = channelVoltageWin * signal.gaussian(winLengthSamples, std=8,sym=False)
       
        # compute fft
        #TODO Populate new 3D array of shape numOfChannelxnumOfWindowsxlen(winAmp) with amp values
        winAmp = abs(scipy.fftpack.rfft(channelVoltageWin,fftLengthSamples)) # determine amplitude spectrum by taking abs
        arrayOfAmps[channelIndex, winIndex,:] = winAmp

        # find peak frequency
        maxAmplitudeIndex = np.argmax(winAmp) # finds simple max amp peak
        true_maxAmplitudeIndex = parabolic(np.log(winAmp), maxAmplitudeIndex-1)[0] # finds parabolic interpolation
        maxFreq = freqs[int(maxAmplitudeIndex)] # retrieves frequency of peak
        true_maxFreq = nyq * true_maxAmplitudeIndex / fftLengthSamples # retrieves frequency of parabolic peak
        print('Channel '+str(channelIndex+1)+ ', Window '+str(winIndex+1)+':     '+str(true_maxFreq)+'  '+str(maxFreq))
        channelPeaks[channelIndex,winIndex] = true_maxFreq # stores peak frequency for every window

Channel 1, Window 1:     10.4498185328  10.5
Channel 1, Window 2:     9.94973331214  10.0
Channel 1, Window 3:     10.1496130284  10.2
Channel 1, Window 4:     10.5499018677  10.6
Channel 1, Window 5:     10.1499252617  10.2
Channel 2, Window 1:     9.34968223803  9.4
Channel 2, Window 2:     9.94966636794  10.0
Channel 2, Window 3:     10.1496502711  10.2
Channel 2, Window 4:     10.5499553528  10.6
Channel 2, Window 5:     10.1498427601  10.2
Channel 3, Window 1:     9.34948729008  9.4
Channel 3, Window 2:     9.94966850585  10.0
Channel 3, Window 3:     10.1496577704  10.2
Channel 3, Window 4:     10.4494004261  10.5
Channel 3, Window 5:     10.1497758156  10.2
Channel 4, Window 1:     9.34947296664  9.4
Channel 4, Window 2:     9.94968518538  10.0
Channel 4, Window 3:     10.1496950034  10.2
Channel 4, Window 4:     10.4496260504  10.5
Channel 4, Window 5:     10.1496587218  10.2
Channel 5, Window 1:     9.34962279307  9.4
Channel 5, Window 2:     9.94973134887  10.0
Channel 5, Win

In [27]:
print np.median(channelPeaks,1)
print np.mean(channelPeaks,1)

[ 10.14992526  10.14965027  10.14965777  10.14965872   9.94973135
   9.59998706   9.59992783  10.09818204]
[ 10.2497984   10.0297594   10.00959796  10.00962759   9.81979573
   9.68964675   9.68961707  10.12916445]


In [28]:
chPeakVariances = []
chPeakMeans = []
chPeakMedians = []
chPeakStandardDeviations = []
for i in range(len(channelPeaks)):
    chPeakVariances.append(np.var(channelPeaks[i,:]))
    chPeakMeans.append(np.mean(channelPeaks[i,:]))
    chPeakMedians.append(np.median(channelPeaks[i,:]))
    chPeakStandardDeviations.append(np.std(channelPeaks[i,:]))

In [29]:
for i in range(len(channelPeaks)):
    print 'Channel ' + str(i+1) + ' Peaks'
    print '\tmean:', chPeakMeans[i]
    print '\tmedian:', chPeakMedians[i]
    print '\tstandard deviation:', chPeakStandardDeviations[i]
    print '\tvariance:', chPeakVariances[i]

Channel 1 Peaks
	mean: 10.2497984006
	median: 10.1499252617
	standard deviation: 0.219144223864
	variance: 0.048024190853
Channel 2 Peaks
	mean: 10.029759398
	median: 10.1496502711
	standard deviation: 0.391999363178
	variance: 0.153663500732
Channel 3 Peaks
	mean: 10.0095979616
	median: 10.1496577704
	standard deviation: 0.366614355106
	variance: 0.13440608537
Channel 4 Peaks
	mean: 10.0096275855
	median: 10.1496587218
	standard deviation: 0.366667004776
	variance: 0.134444692391
Channel 5 Peaks
	mean: 9.81979572579
	median: 9.94973134887
	standard deviation: 0.37630490284
	variance: 0.141605379901
Channel 6 Peaks
	mean: 9.68964674922
	median: 9.59998705651
	standard deviation: 0.3121474633
	variance: 0.0974360388444
Channel 7 Peaks
	mean: 9.68961706593
	median: 9.59992782505
	standard deviation: 0.312137138036
	variance: 0.0974295929411
Channel 8 Peaks
	mean: 10.1291644538
	median: 10.0981820401
	standard deviation: 0.112466863415
	variance: 0.0126487953664


In [30]:
# Do this for all channels, one figure using subplot, each channel with its own subplot
fig4 = plt.figure(4, figsize=(9,9))
numBins = 50

for channelIndex in range(0, numOfChannel):
    plt.subplot(np.ceil(np.sqrt(numOfChannel)),np.ceil(np.sqrt(numOfChannel)),channelIndex)
    n, bins, patches = plt.hist(channelPeaks[channelIndex],numBins,alpha=0.8)
    y = mlab.normpdf( bins, chPeakMeans[channelIndex],chPeakStandardDeviations[channelIndex] )
    l = plt.plot(bins, y, 'r--', linewidth=2)

plt.show()
#for channelIndex in range(0, numOfChannel):
    
#n, bins, patches = plt.hist(channelPeaks[7],numBins,alpha=0.8)

In [34]:
# plot amp for each channel
#for each channel
    #for each window
        #plot 3d figure with time on horizontal, freqs on vertical, and winAmp as the fluctuating value
fig5 = plt.figure(5, figsize=(6,6))

plt.plot(arrayOfAmps[0,1,:])
plt.show()
#print arrayOfAmps[0][0]
#ax = Axes3D(fig5)
ax = plt.axes(projection='3d')
#for channelIndex in range(0, numOfChannel):
#print np.shape(arrayOfAmps[0])
X = range(0,numOfWindows)
y = freqs
X,Y = np.meshgrid(x,y)
X,Y = X.T,Y.T
print np.shape(X)
print np.shape(Y)
Z = arrayOfAmps[0]
print Z
Z = Z.reshape(Y.shape)
ax.plot_surface(X,Y,Z)
plt.show()
#for winIndex in range(0, numOfWindows):
    
    #for lengthOfAmps in range(0, len(freqs)):
         #   plt.subplot(ceil(sqrt(numOfChannel)),ceil(sqrt(numOfChannel)),lengthOfAmps)
#        ax.plot_surface( arrayOfAmps[0], arrayOfAmps[0][winIndex], arrayOfAmps[0][winIndex][lengthOfAmps] ,  cmap=plt.cm.jet, rstride=1, cstride=1, linewidth=0)    

#for zIndex in range(0, len(freqs)):
#   ax.plot(arrayOfAmps[channelIndex][winIndex], arrayOfAmps[channelIndex], arrayOfAmps[channelIndex][winIndex][zIndex])

(5, 5120)
(5, 5120)
[[  1.05471187e-15   3.32542636e-01   4.43021584e-01 ...,   1.08157696e-01
    6.70441611e-02   1.29923202e-01]
 [  1.52655666e-15   9.64062676e-01   2.71795916e+00 ...,   1.94024364e-01
    5.57577640e-02   2.12138290e-01]
 [  8.52096171e-15   1.02154049e+00   3.28433487e+00 ...,   2.08546563e-01
    2.18254819e-01   2.79360810e-01]
 [  1.33226763e-15   8.90778723e-02   1.10697127e+00 ...,   3.92147149e-01
    1.95531666e-02   3.98568705e-01]
 [  7.77156117e-16   1.31678935e-01   6.25674190e-01 ...,   2.60872866e-02
    1.13858311e-03   2.64633103e-02]]
